### Libs 

In [201]:
import pandas as pd
import openpyxl as xl
import schedule 
import time 
import numpy as np
from datetime import datetime,timedelta
from IPython.display import display
import matplotlib.pyplot as plt


### Leitura Dataseat

In [202]:
pd.set_option('display.max_columns', None) # configuração para mostrar todas as colunas do DF


def leitura_excel():
    try:
        # Lê o arquivo CSV
        df = pd.read_csv(r'C:\Users\gm194\Downloads\despesas_contratadas_candidatos_2024_RS.csv', encoding="latin-1", sep=";", nrows=2000)
        display(df.head(1))  # Imprime as primeiras linhas do DataFrame
        return df  # Retorna o DataFrame se a leitura for bem-sucedida
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")  # Tratamento de erros
        return None  # Retorna None se houver um erro

# Executa a função e armazena o DataFrame em uma variável, para poder usar em outras células
df = leitura_excel()

# Verifica se a leitura foi bem-sucedida e exibe o DataFrame completo
if df is not None:
    print("A leitura foi concluída com sucesso.")
else:
    print("A leitura falhou.")

,DT_GERACAO,HH_GERACAO,AA_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,DT_PRESTACAO_CONTAS,SQ_PRESTADOR_CONTAS,SG_UF,SG_UE,NM_UE,NR_CNPJ_PRESTADOR_CONTA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NR_CPF_CANDIDATO,NR_CPF_VICE_CANDIDATO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_TIPO_FORNECEDOR,DS_TIPO_FORNECEDOR,CD_CNAE_FORNECEDOR,DS_CNAE_FORNECEDOR,NR_CPF_CNPJ_FORNECEDOR,NM_FORNECEDOR,NM_FORNECEDOR_RFB,CD_ESFERA_PART_FORNECEDOR,DS_ESFERA_PART_FORNECEDOR,SG_UF_FORNECEDOR,CD_MUNICIPIO_FORNECEDOR,NM_MUNICIPIO_FORNECEDOR,SQ_CANDIDATO_FORNECEDOR,NR_CANDIDATO_FORNECEDOR,CD_CARGO_FORNECEDOR,DS_CARGO_FORNECEDOR,NR_PARTIDO_FORNECEDOR,SG_PARTIDO_FORNECEDOR,NM_PARTIDO_FORNECEDOR,DS_TIPO_DOCUMENTO,NR_DOCUMENTO,CD_ORIGEM_DESPESA,DS_ORIGEM_DESPESA,SQ_DESPESA,DT_DESPESA,DS_DESPESA,VR_DESPESA_CONTRATADA
0,26/10/2024,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,06/10/2024,1,Parcial,13/09/2024,5288288726,RS,86304,AMETISTA DO SUL,56791789000109,13,Vereador,210002297508,12222,LAURY RIBEIRO,-4,-4,12,PDT,Partido Democrático Trabalhista,1,PESSOA JURÍDICA,58298,"Edição integrada à impressão de cadastros, lis...",26709166000175,JULIANO ALBA,JULIANO ALBA,-1,#NULO,#NULO#,-1,#NULO,-1,-1,-1,#NULO,-1,#NULO,#NULO,Nota Fiscal,820,20110000,Publicidade por adesivos,54764890,09/09/2024,ADESIVO MICROPERFURADO,"120,00"


A leitura foi concluída com sucesso.


### Manipulação de datas

In [203]:
# Converte strings para datas. Valores inválidos se tornam NaT( Not a time).
# O parâmetro erros='coerce' é usado para evitar erros na conversão
# Se a data não for válida, ela é transformada em NaT em vez de parar o código.
df['DT_GERACAO'] = pd.to_datetime(df['DT_GERACAO'], errors='coerce', dayfirst=False)
df['DT_ELEICAO'] = pd.to_datetime(df['DT_ELEICAO'], errors='coerce',dayfirst=False)
df['DT_DESPESA'] = pd.to_datetime(df['DT_DESPESA'], errors='coerce',dayfirst=False)

#Após tratar essa coluna, posso passar para string e mudar o formato da data
df['DT_GERACAO'] = df['DT_GERACAO'].dt.strftime('%Y-%m-%d')
df['DT_ELEICAO'] = df['DT_ELEICAO'].dt.strftime('%Y-%m-%d')
df['DT_DESPESA'] = df['DT_DESPESA'].dt.strftime('%Y-%m-%d')

C:\Users\gm194\AppData\Local\Temp\ipykernel_13248\2500760305.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DT_GERACAO'] = pd.to_datetime(df['DT_GERACAO'], errors='coerce', dayfirst=False)


## Tratando valores nulos

In [227]:
df = df.applymap(lambda x: 'Nan' if x == '#NULO' else x)
df = df.applymap(lambda x: 'Nan' if x == '#NULO#' else x)


# df['SG_PARTIDO_FORNECEDOR'] = df['SG_PARTIDO_FORNECEDOR'].replace({'#NULO': np.nan}).str.strip()
# df['NM_PARTIDO_FORNECEDOR'] = df['NM_PARTIDO_FORNECEDOR'].replace({'#NULO': np.nan}).str.strip()
# df['DS_ESFERA_PART_FORNECEDOR'] = df['DS_ESFERA_PART_FORNECEDOR'].replace({'#NULO': np.nan}).str.strip()
# df['SG_UF_FORNECEDOR'] = df['SG_UF_FORNECEDOR'].replace({'#NULO#': np.nan}).str.strip()
# df['NM_MUNICIPIO_FORNECEDOR'] = df['NM_MUNICIPIO_FORNECEDOR'].replace({'#NULO': np.nan}).str.strip()
# df['DS_CARGO_FORNECEDOR'] = df['DS_CARGO_FORNECEDOR'].replace({'#NULO': np.nan}).str.strip()
# df['SG_UF_FORNECEDOR'] = df['SG_UF_FORNECEDOR'].replace({'#NULO': np.nan}).str.strip()

C:\Users\gm194\AppData\Local\Temp\ipykernel_13248\2984418398.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: 'Nan' if x == '#NULO' else x)
C:\Users\gm194\AppData\Local\Temp\ipykernel_13248\2984418398.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: 'Nan' if x == '#NULO#' else x)


## Convertendo colunas para Float | Int

In [205]:
df['VR_DESPESA_CONTRATADA'] = df['VR_DESPESA_CONTRATADA'].str.replace(',' , '.').astype(float)

In [234]:
df.head(1)
df.rename(columns={'DT_GERACAO': 'data_geracao','HH_GERACAO':'hr_geracao','AA_ELEICAO': 'cd_peri',
'CD_TIPO_ELEICAO':'codig_tip_eleicao'})


,data_geracao,hr_geracao,cd_peri,codig_tip_eleicao,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,DT_PRESTACAO_CONTAS,SQ_PRESTADOR_CONTAS,SG_UF,SG_UE,NM_UE,NR_CNPJ_PRESTADOR_CONTA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NR_CPF_CANDIDATO,NR_CPF_VICE_CANDIDATO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_TIPO_FORNECEDOR,DS_TIPO_FORNECEDOR,CD_CNAE_FORNECEDOR,DS_CNAE_FORNECEDOR,NR_CPF_CNPJ_FORNECEDOR,NM_FORNECEDOR,NM_FORNECEDOR_RFB,CD_ESFERA_PART_FORNECEDOR,DS_ESFERA_PART_FORNECEDOR,SG_UF_FORNECEDOR,CD_MUNICIPIO_FORNECEDOR,NM_MUNICIPIO_FORNECEDOR,SQ_CANDIDATO_FORNECEDOR,NR_CANDIDATO_FORNECEDOR,CD_CARGO_FORNECEDOR,DS_CARGO_FORNECEDOR,NR_PARTIDO_FORNECEDOR,SG_PARTIDO_FORNECEDOR,NM_PARTIDO_FORNECEDOR,DS_TIPO_DOCUMENTO,NR_DOCUMENTO,CD_ORIGEM_DESPESA,DS_ORIGEM_DESPESA,SQ_DESPESA,DT_DESPESA,DS_DESPESA,VR_DESPESA_CONTRATADA
0,2024-10-26,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,2024-06-10,1,Parcial,13/09/2024,5288288726,RS,86304,AMETISTA DO SUL,56791789000109,13,Vereador,210002297508,12222,LAURY RIBEIRO,-4,-4,12,PDT,Partido Democrático Trabalhista,1,PESSOA JURÍDICA,58298,"Edição integrada à impressão de cadastros, lis...",26709166000175,JULIANO ALBA,JULIANO ALBA,-1,Nan,Nan,-1,Nan,-1,-1,-1,Nan,-1,Nan,Nan,Nota Fiscal,820,20110000,Publicidade por adesivos,54764890,2024-09-09,ADESIVO MICROPERFURADO,120.0
1,2024-10-26,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,2024-06-10,1,Parcial,13/09/2024,5288288726,RS,86304,AMETISTA DO SUL,56791789000109,13,Vereador,210002297508,12222,LAURY RIBEIRO,-4,-4,12,PDT,Partido Democrático Trabalhista,1,PESSOA JURÍDICA,58298,"Edição integrada à impressão de cadastros, lis...",26709166000175,JULIANO ALBA,JULIANO ALBA,-1,Nan,Nan,-1,Nan,-1,-1,-1,Nan,-1,Nan,Nan,Nota Fiscal,820,20110000,Publicidade por adesivos,54764890,2024-09-09,ADESIVO DE TIRA 30X10,50.0
2,2024-10-26,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,2024-06-10,1,Parcial,13/09/2024,5288288726,RS,86304,AMETISTA DO SUL,56791789000109,13,Vereador,210002297508,12222,LAURY RIBEIRO,-4,-4,12,PDT,Partido Democrático Trabalhista,1,PESSOA JURÍDICA,58298,"Edição integrada à impressão de cadastros, lis...",26709166000175,JULIANO ALBA,JULIANO ALBA,-1,Nan,Nan,-1,Nan,-1,-1,-1,Nan,-1,Nan,Nan,Nota Fiscal,820,20110000,Publicidade por adesivos,54764890,2024-09-09,ADESIVO MICROPERFURADO 100X50,300.0
3,2024-10-26,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,2024-06-10,1,Relatório Financeiro,29/09/2024,5276765724,RS,87858,PASSO FUNDO,56507018000139,13,Vereador,210002140791,45500,EDUARDO VARGAS PELICIOLLI,-4,-4,45,PSDB,Partido da Social Democracia Brasileira,1,PESSOA JURÍDICA,73122,"Agenciamento de espaços para publicidade, exce...",13347016000117,FACEBOOK SERVICOS ONLINE DO BRASIL LTDA,FACEBOOK SERVICOS ONLINE DO BRASIL LTDA.,-1,Nan,Nan,-1,Nan,-1,-1,-1,Nan,-1,Nan,Nan,Fatura,904106620,20420000,Despesa com Impulsionamento de Conteúdos,58134673,2024-05-09,IMPULSIONAMENTO,50.0
4,2024-10-26,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,2024-06-10,1,Relatório Financeiro,29/09/2024,5276765724,RS,87858,PASSO FUNDO,56507018000139,13,Vereador,210002140791,45500,EDUARDO VARGAS PELICIOLLI,-4,-4,45,PSDB,Partido da Social Democracia Brasileira,1,PESSOA JURÍDICA,73122,"Agenciamento de espaços para publicidade, exce...",13347016000117,FACEBOOK SERVICOS ONLINE DO BRASIL LTDA,FACEBOOK SERVICOS ONLINE DO BRASIL LTDA.,-1,Nan,Nan,-1,Nan,-1,-1,-1,Nan,-1,Nan,Nan,Fatura,904103314,20420000,Despesa com Impulsionamento de Conteúdos,58134671,2024-10-09,IMPULSIONAMENTO,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2024-10-26,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,2024-06-10,1,Parcial,10/09/2024,5233408752,RS,86851,GUAÍBA,56068734000167,13,Vereador,210001883461,70005,MARTA BEATRIZ LIMAS BICA,-4,-4,70,AVANTE,AVANTE,1,PESSO

## Conversão de dados

In [228]:
# df_filtrado = df[(df['NM_UE'] == 'PORTO ALEGRE') | (df['NM_UE'] == 'SÃO LEOPOLDO') | (df['NM_UE'] == 'CANOAS')]

# df_filtrado = df_filtrado.groupby('NM_UE')['VR_DESPESA_CONTRATADA'].sum()
# df_filtrado

df.to_excel(r'C:\Users\gm194\Downloads\df.xlsx', index=False)
